# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print('Connection established')
except Exception as e:
    print(e)

Connection established


#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
    
except Exception as e:
        print(e)

#### Set Keyspace

In [7]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query 1
#### Query is done on sessionId and itemInSession. Therefore sessionId is used as partition key and itemInSession is used as clustering column. Columns to be included would be artist, song and song's length.

In [8]:
# Create music_app_hist table for Query 1
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS music_app_hist (
        session_id int, 
        item_in_session int, 
        song text, 
        artist text, 
        length float, 
        PRIMARY KEY(session_id, item_in_session));
    """)
except Exception as e:
    print(e)                   

In [9]:
# Read from csv file and insert into music_app_hist table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute("""
            INSERT INTO music_app_hist 
            (session_id, item_in_session, song, artist, length)
            VALUES (%s, %s, %s, %s, %s);
            """, 
            (int(line[8]), int(line[3]), line[9], line[0], float(line[5]))
        )

In [10]:
## Query rows where sessionId = 338 and itemInSession = 4
rows = session.execute("""
            SELECT song, artist, length
            FROM music_app_hist
            WHERE session_id = 338 AND item_in_session = 4;
        """)
for row in rows:
    print(row)

Row(song='Music Matters (Mark Knight Dub)', artist='Faithless', length=495.30731201171875)


## Query 2
#### Query is done on userId and sessionId sorted by itemInSession. Therefore userId and sessionId is used as the partition key and itemInSession is used as the clustering column. Columns to be included would be artist, song, first name and last name.

In [11]:
# Create user_hist table for Query 2
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_hist (
        session_id int,
        user_id int,
        item_in_session int,
        song text,
        artist text,
        first_name text,
        last_name text,
        PRIMARY KEY ((session_id, user_id), item_in_session))
        WITH CLUSTERING ORDER BY (item_in_session ASC);
    """)
except Exception as e:
    print(e)

In [12]:
# Read from csv file and insert into user_hist table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute("""
            INSERT INTO user_hist 
            (session_id, user_id, item_in_session, song, artist, first_name, last_name) 
            VALUES (%s, %s, %s, %s, %s, %s, %s);
            """,
            (int(line[8]), int(line[10]), int(line[3]), line[9], line[0], line[1], line[4])
        )

In [13]:
# Query rows where sessionId = 182 and userId = 10
rows = session.execute("""
        SELECT song, artist, first_name, last_name
        FROM user_hist
        WHERE session_id = 182 AND user_id = 10;
        """)
for row in rows:
    print(row)

Row(song="Keep On Keepin' On", artist='Down To The Bone', first_name='Sylvie', last_name='Cruz')
Row(song='Greece 2000', artist='Three Drives', first_name='Sylvie', last_name='Cruz')
Row(song='Kilometer', artist='Sebastien Tellier', first_name='Sylvie', last_name='Cruz')
Row(song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', artist='Lonnie Gordon', first_name='Sylvie', last_name='Cruz')


## Query 3
#### Query is done on the song and userId. Therefore song is used as the partition key and userId is used as the clustering column. Columns to be included would be song, first name and last name.

In [14]:
# Create song_hist table for Query 3
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_hist (
        song text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song, user_id))
    """)
except Exception as e:
    print(e)        

In [15]:
# Read from csv file and insert into song_hist table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute("""
            INSERT INTO song_hist
            (song, user_id, first_name, last_name)
            VALUES (%s, %s, %s, %s);
        """,
        (line[9], int(line[10]), line[1], line[4])
        )

In [16]:
# Query rows where song = 'All Hands Against His Own'
rows = session.execute("""
        SELECT song, first_name, last_name
        FROM song_hist
        WHERE song = 'All Hands Against His Own';
        """)
for row in rows:
    print(row)

Row(song='All Hands Against His Own', first_name='Jacqueline', last_name='Lynch')
Row(song='All Hands Against His Own', first_name='Tegan', last_name='Levine')
Row(song='All Hands Against His Own', first_name='Sara', last_name='Johnson')


### Drop the tables

In [18]:
session.execute("DROP TABLE IF EXISTS sparkify.music_app_hist")
session.execute("DROP TABLE IF EXISTS sparkify.user_hist")
session.execute("DROP TABLE IF EXISTS sparkify.song_hist")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()